# Displaying Camera Images

In [ ]:
import matplotlib.pylab as plt
from ctapipe.instrument import SubarrayDescription
from ctapipe.visualization import CameraDisplay
from ctapipe.image import toymodel
from ctapipe.image import hillas_parameters, tailcuts_clean
import numpy as np
import astropy.units as u
import astropy.coordinates as c

First, let's create a fake Cherenkov image from a given `CameraGeometry` and fill it with some data that we can draw later.

In [ ]:
# load an example camera geometry from a simulation file
subarray = SubarrayDescription.read("dataset://gamma_prod5.simtel.zst")
geom = subarray.tel[100].camera.geometry

# create a fake camera image to display:
model = toymodel.Gaussian(
    x=0.2 * u.m,
    y=0.0 * u.m,
    width=0.05 * u.m,
    length=0.15 * u.m,
    psi="35d",
)

image, sig, bg = model.generate_image(geom, intensity=1500, nsb_level_pe=10)

# apply really stupid image cleaning (single threshold):
mask = tailcuts_clean(geom, image, 10, 5)

note: if you want your camera to be displayed as it would be sitting ont he gronud in front of you, you probably want to transform your `CameraGeometry` into the `EngineeringCameraFrame` first.  This is optional. 

In [ ]:
from ctapipe.coordinates import EngineeringCameraFrame

geom = geom.transform_to(EngineeringCameraFrame())

## Displaying Images

The simplest plot is just to generate a CameraDisplay with an image in its constructor.  A figure and axis will be created automatically

In [ ]:
CameraDisplay(geom, image=image)

To change other options, you should call the relevant functions of the `CameraDisplay` object that is returned.  For example to add a color bar, call `add_colorbar()`, or to change the color scale, modify the `cmap` or `norm` properties directly.  

### Changing the color map and scale

In [ ]:
disp = CameraDisplay(geom, image=image)
disp.add_colorbar()
disp.cmap = "rainbow"

By default the minimum and maximum of the color bar are set automatically by the data in the image. To choose fixed limits, use:`

In [ ]:
disp = CameraDisplay(geom, image=image, cmap="rainbow")
disp.add_colorbar()
disp.set_limits_minmax(10, 50)

or as a percentile

In [ ]:
disp = CameraDisplay(geom, image=image, cmap="rainbow")
disp.add_colorbar()
disp.set_limits_percent(50)

### Using different normalizations

You can choose from several preset normalizations (lin, log, symlog) and also provide a custom normalization, for example a `PowerNorm`:

In [ ]:
from matplotlib.colors import PowerNorm

fig, axes = plt.subplots(2, 2, figsize=(10, 10))
norms = ["lin", "log", "symlog", PowerNorm(0.5)]

for norm, ax in zip(norms, axes.flatten()):
    disp = CameraDisplay(geom, image=image, ax=ax)
    disp.norm = norm
    ax.set_title(str(norm))

axes[1, 1].set_title("PowerNorm(0.5)")
plt.show()

## Overlays

### Marking pixels

here we will mark pixels in the image mask.  That will change their outline color

In [ ]:
disp = CameraDisplay(geom, image=image, cmap="gray")
disp.highlight_pixels(mask, alpha=0.5, color="red")

### Drawing a Hillas-parameter ellipse

For this, we will first compute some Hillas Parameters in the current frame:

In [ ]:
clean_image = image.copy()
clean_image[~mask] = 0
hillas = hillas_parameters(geom, clean_image)

plt.figure(figsize=(10, 10))
disp = CameraDisplay(geom, image=image, cmap="gray_r")
disp.overlay_moments(hillas, color="orange", linewidth=3, zorder=10)

### Drawing a marker at a coordinate

This depends on the coordinate frame of the `CameraGeometry`.  Here we will sepcify the coordinate the `EngineerngCameraFrame`, but if you have enough information to do the coordinate transform, you could use `ICRS` coordinates and overlay star positions. 

In [ ]:
plt.figure(figsize=(6, 6))
disp = CameraDisplay(geom, image=image, cmap="gray_r")

coord = c.SkyCoord(x=0.5 * u.m, y=0.7 * u.m, frame=EngineeringCameraFrame())
disp.overlay_coordinate(coord, markersize=20)

## Using CameraDisplays interactively

`CameraDisplays` can be used interactivly whe displayed in a window, and also when using Jupyter notebooks/lab with appropriate backends. 

When this is the case, the same `CameraDisplay` object can be re-used.  We can't show this here in the documentation, but creating an animation when in a matplotlib window is quite easy! Try this in an interactive ipython session:

### Running interactive displays in a matplotlib window

```sh
ipython -i --maplotlib=auto
```

That will open an ipython session with matplotlib graphics in a separate thread, meaning that you can type code and interact with plots simultaneneously. 

In the ipython session try running the following code and you will see an animation (here in the documentation, it will of course be static)

First we load some real data so we have a nice image to view:

In [ ]:
import matplotlib.pyplot as plt
from ctapipe.io import EventSource
from ctapipe.visualization import CameraDisplay
import numpy as np

DATA = "dataset://gamma_20deg_0deg_run1___cta-prod5-lapalma_desert-2158m-LaPalma-dark_100evts.simtel.zst"

with EventSource(
    DATA,
    max_events=1,
    focal_length_choice="EQUIVALENT",
) as source:
    event = next(iter(source))

tel_id = list(event.r0.tel.keys())[0]
geom = source.subarray.tel[tel_id].camera.geometry
waveform = event.r0.tel[tel_id].waveform
n_chan, n_pix, n_samp = waveform.shape

Running the following the will bring up a window and animate the shower image as a function of time. 

In [ ]:
disp = CameraDisplay(geom)

for ii in range(n_samp):
    disp.image = waveform[0, :, ii]
    plt.pause(0.1)  # this lets matplotlib re-draw the scene

### Making it clickable 

Also when running in a window, you can enable the `disp.enable_pixel_picker()` option.  This will then allow the user to click a pixel and a function will run.  By default the function simply prints the pixel and value to stdout, however you can override the function `on_pixel_clicked(pix_id)` to do anything you want by making a subclass

In [ ]:
class MyCameraDisplay(CameraDisplay):
    def on_pixel_clicked(self, pix_id):
        print(f"{pix_id=} has value {self.image[pix_id]:.2f}")

In [ ]:
disp = MyCameraDisplay(geom, image=image)
disp.enable_pixel_picker()

then, whena user clicks a pixel it would print:
```
pixel 5 has value 2.44
```

## A continously running camera display

Try the following code outside of a notebook to get a contiously updating camera display

```python
!/usr/bin/env python3
"""
Example of drawing and updating a Camera using a toymodel shower images.

the animation should remain interactive, so try zooming in when it is
running.
"""

import matplotlib.pylab as plt
import numpy as np
from astropy import units as u
from matplotlib.animation import FuncAnimation

from ctapipe.image import toymodel
from ctapipe.instrument import SubarrayDescription
from ctapipe.visualization import CameraDisplay

if __name__ == "__main__":

    fig, ax = plt.subplots()

    # load the camera
    subarray = SubarrayDescription.read("dataset://gamma_prod5.simtel.zst")
    geom = subarray.tel[1].camera.geometry

    fov = 0.3
    maxwid = 0.05
    maxlen = 0.1

    disp = CameraDisplay(geom, ax=ax)
    disp.cmap = "inferno"
    disp.add_colorbar(ax=ax)

    def update(frame):
        x, y = np.random.uniform(-fov, fov, size=2)
        width = np.random.uniform(0.01, maxwid)
        length = np.random.uniform(width, maxlen)
        angle = np.random.uniform(0, 180)
        intens = width * length * (5e4 + 1e5 * np.random.exponential(2))

        model = toymodel.Gaussian(
            x=x * u.m,
            y=y * u.m,
            width=width * u.m,
            length=length * u.m,
            psi=angle * u.deg,
        )
        image, _, _ = model.generate_image(
            geom,
            intensity=intens,
            nsb_level_pe=5,
        )
        disp.image = image

    anim = FuncAnimation(fig, update, interval=500)
    plt.show()
```

# Other features

## Plotting with Bokeh instead of Matplotlib

There is a second experimental implementation of CameraDisplay that uses Bokeh as a backend.  Use it as follows:

In [ ]:
from ctapipe.visualization.bokeh import CameraDisplay
import bokeh

bokeh.io.output_notebook()

In [ ]:
bdisp = CameraDisplay(geom, image=image)
bdisp.show()